In [1]:
from pyspark.sql import SparkSession

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [2]:
# Create a spark session
spark = SparkSession.builder.appName("HAI-Preprocessing").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/07 16:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/07 16:59:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Creating  a spark dataframe
train1 = spark.read.csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/hai-train1.txt", header=True, inferSchema=True)

In [18]:
# Credentials
token = "UPFkGSmmgXErDnl75fW1FeqUfQ0j6BsQWcff0uEWuQUJHjTq2LtTuAL9nhjkgp-5xtSo6KQsS60_1QJCamiyxQ=="
org = "OSTProject"
url = "http://localhost:8086"

In [19]:
# Select the rows
test = train1.limit(10)

# Create a new DataFrame  
new_dataframe = spark.createDataFrame(test.rdd, train1.schema)

In [20]:
# Bucket
bucket = "StatData"

# Initialize Client
with InfluxDBClient(url=url, token=token, org=org) as client:
    # Write Data
    write_api = client.write_api(write_options=SYNCHRONOUS)
    
    data = test.collect()

    for row in data:
        point = Point("HAI_measures")

        # Add field
        for col in test.columns[1:]:
            point.field(col, row[col])

        print(point)
        write_api.write(bucket, org, point)

HAI_measures P1_FCV01D=72.15386,P1_FCV01Z=72.59521,P1_FCV02D=15,P1_FCV02Z=12.44507,P1_FCV03D=52.55638,P1_FCV03Z=53.76587,P1_FT01=54.96975,P1_FT01Z=293.16074,P1_FT02=2460.51758,P1_FT02Z=3162.2229,P1_FT03=238.83813,P1_FT03Z=982.6814,P1_LCV01D=11.12804,P1_LCV01Z=11.23962,P1_LIT01=346.73645,P1_PCV01D=27.86273,P1_PCV01Z=28.64227,P1_PCV02D=12,P1_PCV02Z=11.81946,P1_PIT01=1.26053,P1_PIT01_HH=524320i,P1_PIT02=0.37628,P1_PP01AD=0i,P1_PP01AR=0i,P1_PP01BD=1i,P1_PP01BR=1i,P1_PP02D=1i,P1_PP02R=1i,P1_PP04=33.00217,P1_PP04D=1i,P1_PP04SP=27.98469,P1_SOL01D=524320i,P1_SOL03D=524320i,P1_STSP=1i,P1_TIT01=30.18799,P1_TIT02=35.19287,P1_TIT03=27.22479,P2_24Vdc=28.03777,P2_ATSW_Lamp=1i,P2_AutoGO=1i,P2_AutoSD=112,P2_Emerg=0i,P2_MASW=0i,P2_MASW_Lamp=0i,P2_ManualGO=0i,P2_ManualSD=0,P2_OnOff=1i,P2_RTR=2880i,P2_SCO=54741i,P2_SCST=720.797,P2_SIT01=917i,P2_TripEx=1i,P2_VIBTR01=6.4039,P2_VIBTR02=-2.4945,P2_VIBTR03=9.3018,P2_VIBTR04=-0.2176,P2_VT01=11.98277,P2_VTR01=50i,P2_VTR02=50i,P2_VTR03=50i,P2_VTR04=50i,P3_FIT01=

In [21]:
# Initialize Client
client = InfluxDBClient(url=url, token=token, org=org)

# Query
query_api = client.query_api()

query = """from(bucket: "StatData")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "HAI_measures")"""
tables = query_api.query(query, org="OSTProject")

for table in tables:
  for record in table.records:
    print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 12, 7, 16, 50, 58, 537573, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 12, 7, 17, 0, 58, 537573, tzinfo=tzutc()), '_time': datetime.datetime(2023, 12, 7, 17, 0, 56, 71597, tzinfo=tzutc()), '_value': 72.15386, '_field': 'P1_FCV01D', '_measurement': 'HAI_measures'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 12, 7, 16, 50, 58, 537573, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 12, 7, 17, 0, 58, 537573, tzinfo=tzutc()), '_time': datetime.datetime(2023, 12, 7, 17, 0, 56, 149816, tzinfo=tzutc()), '_value': 72.13303, '_field': 'P1_FCV01D', '_measurement': 'HAI_measures'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 12, 7, 16, 50, 58, 537573, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 12, 7, 17, 0, 58, 537573, tzinfo=tzutc()), '_time': datetime.datetime(2023, 12, 7, 17, 0, 56, 161544, tzinfo=